In [ ]:
# @title Call: Install Libraries

# Purpose:
# Install required libraries

!pip install pandas numpy scikit-learn scipy plotly pillow

In [ ]:
# @title Call: Import Libraries

# Purpose:
# Import required libraries

# Pandas & Numpy for Data Processing
import pandas as pd
import numpy as np

# Sklearn & Scipy for Model Creation
from sklearn.cluster import DBSCAN
from scipy.stats import gaussian_kde

# Plotly & PIL for Charting
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from PIL import Image

# base64 for Image Processing
import base64

# Other
import warnings

In [ ]:
# @title Call: Configure Libraries

# Purpose:
# Configure Libraries

pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

Mounted at /content/drive


In [ ]:
# @title Function: load_model_data()

# Purpose:
# Loads data to model

def load_model_data(data_path):

    data = pd.read_excel(data_path, sheet_name=None)

    Instance_df = data['Instance-ID']       # Mapping Table:  Instance Name : Instance ID
    Zone_df = data['Zone-ID']               # Mapping Table:  Zone Name : Zone ID
    Class_df = data['Class-ID']             # Mapping Table:  Class Name : Class ID
    Race_df = data['Race-ID']               # Mapping Table:  Race Name : Race ID
    NPC_df = data['NPC-ID']                 # Mapping Table:  NPC Name : NPC ID
    NPC_Data_df = data['NPC-Data']          # Data Table:     NPC Data
    Death_Data_df = data['Entries_Clean']   # Data Table:     Death Data

    return Instance_df, Zone_df, Class_df, Race_df, NPC_df, NPC_Data_df, Death_Data_df

# Usage example:
# Instance_df, Zone_df, Class_df, Race_df, NPC_df, NPC_Data_df, Death_Data_df = load_model_data("your_data_path.xlsx")

In [ ]:
# @title Function: load_region_analysis()

# Purpose:
# Loads region analysis into model

def load_region_analysis(region_summary_path):

  data = pd.read_excel(region_summary_path, sheet_name='Zone Summary')

  return data

# Usage example:
# load_region_analysis("your_data_path.xlsx")

In [ ]:
# @title Function: format_Death_Data_df()

# Purpose:
# Reformats Death Record Entries as a Pivoted Dataframe

def format_Death_Data_df(Death_Data_df):

    df = Death_Data_df.pivot(index=['Server', 'Character Name', 'Character ID'], columns='Field', values='Field Value')
    df.reset_index(inplace=True)

    Death_Data_df_pivot = df[['Server', 'Character Name', 'Character ID', 'class_id', 'date', 'guild', 'last_words', 'level', 'map_id', 'map_pos', 'race_id', 'source_id']]

    return Death_Data_df_pivot

# Usage example:
# Death_Data_df = format_Death_Data_df(Death_Data_df)

In [ ]:
# @title Function: drop_na_rows()

# Purpose:
# Drop Missing records from Death_Data_df

def drop_na_rows(df, column):

    df_dropna = df.dropna(subset=[column])

    return df_dropna

# Usage example:
# Death_Data_df = drop_na_rows(Main_df, 'map_id')

In [ ]:
# @title Function: create_Main_df()

# Purpose:
# Create Main dataframe

def create_Main_df(Death_Data_df, Zone_df, Race_df, Class_df, NPC_df, NPC_Data_df):

    Main_df = Death_Data_df

    zone_map = Zone_df.set_index('Zone ID')['Zone'].to_dict()
    Main_df['map_name'] = Main_df['map_id'].apply(lambda x: zone_map.get(int(x), x))

    race_map = Race_df.set_index('Race ID')['Race'].to_dict()
    Main_df['race_name'] = Main_df['race_id'].apply(lambda x: race_map.get(int(x), x))

    class_map = Class_df.set_index('Class ID')['Class'].to_dict()
    Main_df['class_name'] = Main_df['class_id'].apply(lambda x: class_map.get(int(x), x))

    npc_map = NPC_df.set_index('NPC ID')['NPC'].to_dict()
    Main_df['npc_name'] = Main_df['source_id'].apply(lambda x: npc_map.get(int(x), ""))

    npc_data_map_name = NPC_Data_df.set_index('ID')['Name'].to_dict()
    npc_data_map_start_level = NPC_Data_df.set_index('ID')['Start Level'].to_dict()
    npc_data_map_end_level = NPC_Data_df.set_index('ID')['End Level'].to_dict()
    npc_data_map_elite = NPC_Data_df.set_index('ID')['Elite'].to_dict()
    npc_data_map_rare = NPC_Data_df.set_index('ID')['Rare'].to_dict()
    npc_data_map_boss = NPC_Data_df.set_index('ID')['Boss'].to_dict()
    npc_data_map_type = NPC_Data_df.set_index('ID')['Type'].to_dict()

    Main_df['npc_name'] = Main_df['source_id'].apply(lambda x: npc_data_map_name.get(int(x), ""))
    Main_df['npc_start_level'] = Main_df['source_id'].apply(lambda x: npc_data_map_start_level.get(int(x), ""))
    Main_df['npc_end_level'] = Main_df['source_id'].apply(lambda x: npc_data_map_end_level.get(int(x), ""))
    Main_df['npc_elite_status'] = Main_df['source_id'].apply(lambda x: npc_data_map_elite.get(int(x), ""))
    Main_df['npc_rare_status'] = Main_df['source_id'].apply(lambda x: npc_data_map_rare.get(int(x), ""))
    Main_df['npc_boss_status'] = Main_df['source_id'].apply(lambda x: npc_data_map_boss.get(int(x), ""))
    Main_df['npc_type'] = Main_df['source_id'].apply(lambda x: npc_data_map_type.get(int(x), ""))

    return Main_df

# Usage example:
# Main_df = create_Main_df(Main_df, Zone_df, Race_df, Class_df, NPC_df, NPC_Data_df)

In [ ]:
# @title Function: clean_Main_df()

def clean_Main_df(Main_df):
    Main_df['level'] = pd.to_numeric(Main_df['level'], errors='coerce')

    Main_df['level_range'] = pd.np.where(Main_df['level'] <= 9, '1-9',
                                         pd.np.where(Main_df['level'] <= 19, '10-19',
                                                     pd.np.where(Main_df['level'] <= 29, '20-29',
                                                                 pd.np.where(Main_df['level'] <= 39, '30-39',
                                                                             pd.np.where(Main_df['level'] <= 49, '40-49',
                                                                                         pd.np.where(Main_df['level'] <= 59, '50-59', '60')
                                                                                        )
                                                                        )
                                                            )
                                                )
                                    )

    Main_df['npc_start_level'] = pd.to_numeric(Main_df['npc_start_level'], errors='coerce')
    Main_df['npc_end_level'] = pd.to_numeric(Main_df['npc_end_level'], errors='coerce')
    Main_df['npc_avg_level'] = round((Main_df['npc_start_level'] + Main_df['npc_end_level']) / 2)
    Main_df['npc_less_player_level'] = Main_df['npc_avg_level'] - Main_df['level']

    Main_df['date'] = pd.to_datetime(Main_df['date'], unit='s')
    Main_df['map_pos'] = Main_df['map_pos'].str.strip('"')
    Main_df[['x', 'y']] = Main_df['map_pos'].str.split(',', expand=True).astype(float)
    Main_df['y'] = Main_df['y'].apply(lambda y: 1 - y)
    Main_df['last_words'] = Main_df['last_words'].str.replace('"', '')
    Main_df['guild'] = Main_df['guild'].str.replace('"', '')

    return Main_df

# Usage example:
# Main_df = clean_Main_df(Main_df)

In [ ]:
# @title Function: define_class_colors()

# Purpose:
# Define Class Colors

def define_class_colors():

    class_colors = {
        'Druid': '#FF7D0A',      # Orange
        'Hunter': '#ABD473',     # Green
        'Mage': '#69CCF0',       # Light blue
        'Paladin': '#F58CBA',    # Pink
        'Priest': '#FFFFFF',     # White
        'Rogue': '#FFF569',      # Light yellow
        'Shaman': '#0070DE',     # Blue
        'Warlock': '#9482C9',    # Purple
        'Warrior': '#C79C6E',    # Tan
        'Total':  '#000000'      # Black
    }
    return class_colors

# Usage example:
# class_colors = define_class_colors()

In [ ]:
# @title Function: define_race_colors()

# Purpose:
# Define Race Colors

def define_race_colors():

    race_colors = {
        'Human': '#006699',      # Blue
        'Orc': '#FF4400',        # Orange
        'Dwarf': '#FFCC33',      # Yellow
        'Night Elf': '#9933FF',  # Purple
        'Undead': '#660066',     # Dark Purple
        'Tauren': '#C79C6E',     # Tan
        'Gnome': '#66FFCC',      # Cyan
        'Troll': '#336600'       # Green
    }
    return race_colors

# Usage example:
# race_colors = define_race_colors()

In [ ]:
# @title Function: create_count_table()

# Purpose:
# Create Count Table for Visuals

def create_count_table(Main_df):

    count_table = pd.pivot_table(Main_df, index='class_name', columns='level', aggfunc='size', fill_value=0)
    total_row = count_table.sum(axis=0)
    total_row.name = 'Total'
    count_table = count_table.append(total_row)

    return count_table

# Usage example:
# count_table = create_count_table(Main_df)

In [ ]:
# @title Function: create_players_alive_by_level_df()

def create_players_alive_by_level_df(count_table):

    percent_table = count_table.copy()
    percent_table = percent_table.div(percent_table.sum(axis=1), axis=0).mul(100)
    percent_table = 100 - percent_table.cumsum(axis=1).round(2)

    players_alive_by_level_df = percent_table

    return players_alive_by_level_df

# Usage example:
# players_alive_by_level_df = create_players_alive_by_level_df(count_table)

In [ ]:
# @title Visual: race_summary_chart()

# Purpose:
# Create Character Race Summary Chart

def race_summary_chart(Main_df, race_colors):

    # Create a new DataFrame with race_name counts
    summary_df = Main_df['race_name'].value_counts().reset_index()
    summary_df.columns = ['race_name', 'count']

    # Calculate the percentage of total records
    total_records = len(Main_df)
    summary_df['percentage'] = (summary_df['count'] / total_records) * 100

    # Create the bar chart using Plotly
    fig = go.Figure(data=[
        go.Bar(
            x=summary_df['race_name'],
            y=(summary_df['count'] / total_records),  # Set y-axis as percentage
            marker_color=[race_colors.get(c, '#000000') for c in summary_df['race_name']],
            text=[f"{count} ({percentage:.2f}%)" for count, percentage in zip(summary_df['count'], summary_df['percentage'])],  # Add count and percentage as text
            textposition='auto',  # Set text position to be automatically determined
            hovertemplate='%{text}',  # Customize the hover template
            texttemplate='%{text}'  # Display the text label on the bars
        )
    ])

    fig.update_layout(
        title='Total Deaths Recorded by Race',
        xaxis_title='Race Name',
        yaxis_title='Percentage of Total Records',
        yaxis_tickformat='.0f%',  # Format y-axis tick labels as percentage with no decimal places and "%" symbol
        plot_bgcolor='rgba(0, 0, 0, 0)'  # Set plot background color to transparent
    )

    fig.show()

# Usage example:
# race_summary_chart(Main_df, race_colors)

In [ ]:
# @title Visual: class_summary_chart()

# Purpose:
# Create Character Class Summary Chart

def class_summary_chart(Main_df, class_colors):

    # Create a new DataFrame with class_name counts
    summary_df = Main_df['class_name'].value_counts().reset_index()
    summary_df.columns = ['class_name', 'count']

    # Calculate the percentage of total records
    total_records = len(Main_df)
    summary_df['percentage'] = (summary_df['count'] / total_records) * 100

    # Create the bar chart using Plotly
    fig = go.Figure(data=[
        go.Bar(
            x=summary_df['class_name'],
            y=(summary_df['count'] / total_records),  # Set y-axis as percentage
            marker_color=[class_colors.get(c, '#000000') for c in summary_df['class_name']],
            text=[f"{count} ({percentage:.2f}%)" for count, percentage in zip(summary_df['count'], summary_df['percentage'])],  # Add count and percentage as text
            textposition='auto',  # Set text position to be automatically determined
            hovertemplate='%{text}',  # Customize the hover template
            texttemplate='%{text}'  # Display the text label on the bars
        )
    ])

    fig.update_layout(
        title='Total Deaths Recorded by Class',
        xaxis_title='Class Name',
        yaxis_title='Percentage of Total Records',
        yaxis_tickformat='.0f%',  # Format y-axis tick labels as percentage with no decimal places and "%" symbol
        plot_bgcolor='rgba(0, 0, 0, 0)'  # Set plot background color to transparent
    )

    fig.show()

# Usage example:
# class_summary_chart(Main_df, class_colors)

In [ ]:
# @title Visual: level_summary_chart()

# Purpose:
# Create Character Level Summary Chart

def level_summary_chart(Main_df):

    # Create a new DataFrame with level counts
    summary_df = Main_df['level'].value_counts().reset_index()
    summary_df.columns = ['level', 'count']

    # Calculate the percentage of total records
    total_records = len(Main_df)
    summary_df['percentage'] = (summary_df['count'] / total_records) * 100

    fig = go.Figure(data=[
        go.Bar(
            x=summary_df['level'],
            y=(summary_df['count'] / total_records),  # Set y-axis as percentage
            text=[f"Level {level}: {count} ({percentage:.2f}%)" for level, count, percentage in zip(summary_df['level'], summary_df['count'], summary_df['percentage'])],  # Add level, count, and percentage as text
            textposition='auto',  # Set text position to be automatically determined
            hovertemplate='%{text}',  # Customize the hover template
            texttemplate='%{text}',  # Display the text label on the bars
            marker_color='#D3D3D3'  # Set the bar color as light gray
        )
    ])

    fig.update_layout(
        title='Total Deaths Recorded by Level',
        xaxis=dict(
            title='Level',
            tickmode='linear',  # Set tick mode to linear
            dtick=1,  # Set tick interval to 1
            tickfont=dict(size=10)  # Set tick font size
        ),
        xaxis_title='Level',
        yaxis_title='Percentage of Total Records',
        yaxis_tickformat='.0f%',  # Format y-axis tick labels as percentage with no decimal places and "%" symbol
        plot_bgcolor='rgba(0, 0, 0, 0)'  # Set plot background color to transparent
    )

    fig.show()

# Usage example:
# level_summary_chart(Main_df)

In [ ]:
# @title Visual: zone_summary_chart()

# Purpose:
# Create Zone Summary Chart

def zone_summary_chart(Main_df):

    # Create a new DataFrame with map_name counts
    summary_df = Main_df['map_name'].value_counts().reset_index()
    summary_df.columns = ['map_name', 'count']

    # Calculate the percentage of total records
    total_records = len(Main_df)
    summary_df['percentage'] = (summary_df['count'] / total_records) * 100

    fig = go.Figure(data=[
        go.Bar(
            x=summary_df['map_name'],
            y=(summary_df['count'] / total_records),  # Set y-axis as percentage
            text=[f"Map: {name}\nCount: {count} ({percentage:.2f}%)" for name, count, percentage in zip(summary_df['map_name'], summary_df['count'], summary_df['percentage'])],  # Add map name, count, and percentage as text
            textposition='auto',  # Set text position to be automatically determined
            hovertemplate='%{text}',  # Customize the hover template
            texttemplate='%{text}',  # Display the text label on the bars
            marker_color='#D3D3D3'  # Set the bar color as light gray
        )
    ])

    fig.update_layout(
        title='Total Deaths Recorded by Zone',
        xaxis_title='Map Name',
        yaxis_title='Percentage of Total Records',
        yaxis_tickformat='.0f%',  # Format y-axis tick labels as percentage with no decimal places and "%" symbol
        plot_bgcolor='rgba(0, 0, 0, 0)'  # Set plot background color to transparent
    )

    fig.show()

# Usage example:
# zone_summary_chart(Main_df)

In [ ]:
# @title Visual: top_death_sources_by_class()

# Purpose:
# Create Top Death Sources by Class Chart

def top_death_sources_by_class(Main_df, class_colors, top_n):

    # Get the top npc_names per class
    top_npc_names = Main_df.groupby(['class_name', 'npc_name']).size().groupby(level=0).nlargest(top_n).reset_index(level=0, drop=True).reset_index(name='count')

    # Create a new DataFrame with npc_name counts by class
    summary_df = top_npc_names.pivot(index='npc_name', columns='class_name', values='count').fillna(0).reset_index()

    # Sort the summary_df by the count for each class separately
    sorted_dfs = {}
    for column in summary_df.columns[1:]:
        sorted_dfs[column] = summary_df[['npc_name', column]].sort_values(column, ascending=False)

    # Create subplots for each class
    fig = make_subplots(rows=len(summary_df.columns[1:]), cols=1, subplot_titles=summary_df.columns[1:], shared_xaxes=True, vertical_spacing=0.02)

    for i, column in enumerate(summary_df.columns[1:], start=1):
        sorted_df = sorted_dfs[column]
        non_zero_df = sorted_df[sorted_df[column] > 0]  # Filter out NPCs with zero records

        fig.add_trace(
            go.Bar(
                x=non_zero_df[column],
                y=non_zero_df['npc_name'],
                orientation='h',
                name=column,
                text=[f'{npc_name}<br>Total Records: {count}' for npc_name, count in zip(non_zero_df['npc_name'], non_zero_df[column])],
                hovertemplate='%{text}',
                marker_color=class_colors.get(column, '#000000')  # Color code the bars based on class_colors
            ),
            row=i,
            col=1
        )

        fig.update_yaxes(title_text='NPC Name', row=i, col=1)
        fig.update_xaxes(title_text='Total Records', row=i, col=1)

    fig.update_layout(
        title='Top Death Sources by Class',
        height=400 * len(summary_df.columns[1:]),  # Adjust the height based on the number of classes
        plot_bgcolor='rgba(0, 0, 0, 0)',  # Set plot background color to transparent
        showlegend=False,
    )

    fig.show()

# Usage example:
# top_death_sources_by_class(Main_df, class_colors, 3)

In [ ]:
# @title Visual: deaths_by_class_and_level_chart()

# Purpose:
# Create Deaths by Class & Level Chart

def deaths_by_class_and_level_chart(count_table, class_colors):

    # Create a list to hold the traces
    traces = []

    # Iterate over each class
    for class_name in count_table.index:
        # Get the data for the current class
        data = count_table.loc[class_name]

        # Create a trace for the current class
        trace = go.Scatter(
            x=data.index,
            y=data.values,
            mode='lines',
            name=class_name,
            line=dict(color=class_colors[class_name]),
            hovertemplate='Class: ' + class_name + '<br>Level: %{x}<br>Count: %{y}<extra></extra>'
        )

        # Add the trace to the list
        traces.append(trace)

    # Create the layout
    layout = go.Layout(
        title='Total Deaths Recorded by Class by Level',
        xaxis=dict(title='Level'),
        yaxis=dict(title='Count'),
        hovermode='closest',
        legend=dict(title='Class'),
        showlegend=True
    )

    # Create the figure
    fig = go.Figure(data=traces, layout=layout)

    # Show the figure
    fig.show()

# Usage example:
# deaths_by_class_and_level_chart(create_count_table(Main_df), class_colors)

In [ ]:
# @title Visual: percentage_players_alive_by_level()

# Purpose:
# Create Percentage of Players Alive by Level Chart

def percentage_players_alive_by_level(players_alive_by_level_df, class_colors):

    # Convert the percent_table DataFrame to a long format
    df = players_alive_by_level_df.reset_index().melt(id_vars='class_name', var_name='level', value_name='percentage')

    # Create a list to hold the traces
    traces = []

    # Iterate over each class
    for class_name in df['class_name'].unique():
        # Filter the data for the current class
        filtered_data = df[df['class_name'] == class_name]

        # Create a trace for the current class
        trace = go.Scatter(
            x=filtered_data['level'],
            y=filtered_data['percentage'],
            mode='lines',
            name=class_name,
            line=dict(color=class_colors[class_name], width=2),
            hovertemplate='Class: ' + class_name + '<br>Level: %{x}<br>Percentage: %{y:.2f}%<extra></extra>'
        )

        # Add the trace to the list
        traces.append(trace)

    # Create the layout
    layout = go.Layout(
        title='Percentage of Players Alive by Level',
        xaxis=dict(title='Level'),
        yaxis=dict(title='Percentage'),
        hovermode='closest',
        legend=dict(title='Class'),
        showlegend=True
    )

    # Create the figure
    fig = go.Figure(data=traces, layout=layout)

    # Show the figure
    fig.show()

# Usage example:
# percentage_players_alive_by_level(players_alive_by_level_df, class_colors)

In [ ]:
# @title Visual: pre_clustering_heatmap_visual()

# Purpose:
# Create Pre-Clustering Heatmap Visual

def pre_clustering_heatmap_visual(Map_Name, Main_df):
    # Read the image file and encode it to base64
    with open(f'{map_data_path}{Map_Name}.jpg', 'rb') as image_file:
        encoded_image = base64.b64encode(image_file.read()).decode()

    # Get the image dimensions
    with Image.open(image_file.name) as img:
        image_width, image_height = img.size

    # Filter the DataFrame based on the map name
    filtered_df = Main_df[Main_df['map_name'] == Map_Name]

    # Create the main map figure
    fig = go.Figure()

    # Add the main map image as a layout image
    fig.add_layout_image(
        source='data:image/jpeg;base64,' + encoded_image,
        x=0,
        y=1,
        sizex=1,
        sizey=1,
        sizing='stretch',
        opacity=1,
        layer='below'
    )

    # Scale the x and y coordinates based on the image dimensions
    scaled_x = filtered_df['x'] * image_width
    scaled_y = (1 - filtered_df['y']) * image_height

    # Add all the points as a scatter plot
    fig.add_trace(go.Scatter(
        x=scaled_x,
        y=scaled_y,
        mode='markers',
        marker=dict(
            size=10,
            opacity=0.5,
        ),
        showlegend=False
    ))

    # Configure the layout with adjusted axes ranges
    fig.update_layout(
        width=800,
        height=600,
        xaxis=dict(visible=False, range=[0, image_width]),
        yaxis=dict(visible=False, range=[image_height, 0]),  # Invert the y-axis range
        margin=dict(l=0, r=0, t=0, b=0)  # Set the margins to 0 on all sides
    )

    # Show the figure
    fig.show()

# Usage example:
# pre_clustering_heatmap_visual(input("Enter the map name: "), Main_df)

In [ ]:
# @title Visual: create_heatmap()

# Purpose:
# Create Heatmap for Region Visual

def create_heatmap(Main_df, map_data_path, map_name, class_colors):
    try:
        # Read the image file and encode it to base64
        file_path = f'{map_data_path}{map_name}.jpg'

        with open(file_path, 'rb') as image_file:
            encoded_image = base64.b64encode(image_file.read()).decode()

        # Get the image dimensions
        with Image.open(image_file.name) as img:
            image_width, image_height = img.size

        # Filter the DataFrame based on the map name
        filtered_df = Main_df[Main_df['map_name'] == map_name]

        # Perform DBSCAN clustering on the scaled coordinates (x, y)
        X = filtered_df[['x', 'y']]
        eps = 0.02
        min_samples_percent = 0.0175
        min_samples = max(int(len(X) * min_samples_percent), 2)
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        cluster_labels = dbscan.fit_predict(X)

        # Add the cluster labels as a column in the DataFrame
        filtered_df['cluster_label'] = cluster_labels

        # Create the main map figure
        fig = go.Figure()

        # Add the main map image as a layout image
        fig.add_layout_image(
            source='data:image/jpeg;base64,' + encoded_image,
            x=0,
            y=1,
            sizex=1,
            sizey=1,
            sizing='stretch',
            opacity=1,
            layer='below'
        )

        # Iterate over unique cluster labels
        for cluster_label in filtered_df['cluster_label'].unique():
            if cluster_label == -1:
                continue

            # Filter the DataFrame for the current cluster label
            cluster_df = filtered_df[filtered_df['cluster_label'] == cluster_label]

            # Scale the x and y coordinates based on the image dimensions
            scaled_x = cluster_df['x'] * image_width
            scaled_y = (1 - cluster_df['y']) * image_height

            # Determine the opacity and color for the clusters
            opacity = 0.1 if cluster_label >= 0 else 0.5
            color = 'lightgray' if cluster_label == -1 else None

            # Calculate cluster center
            center_x = scaled_x.mean()
            center_y = scaled_y.mean()

            # Create hover text with cluster details
            hover_text = (
                f"<b>{cluster_df['npc_name'].mode().values[0]}</b><br>"
                f"<b>lvl: {cluster_df['level'].median():.0f}</b><br>"
            )

            # Add the clustered records as a scatter plot with hover text
            fig.add_trace(go.Scatter(
                x=scaled_x,
                y=scaled_y,
                mode='markers',
                marker=dict(
                    size=10,
                    opacity=opacity,
                    color=color,
                ),
                text=hover_text,
                hoverinfo='text',
                showlegend=False
            ))

            # Add label for cluster center
            fig.add_annotation(
                x=center_x,
                y=center_y,
                xref='x',
                yref='y',
                text=hover_text,
                showarrow=True,
                arrowhead=1,
                ax=0,
                ay=-30
            )

        # Configure the layout with adjusted axes ranges
        fig.update_layout(
            width=800,
            height=600,
            xaxis=dict(visible=False, range=[0, image_width]),
            yaxis=dict(visible=False, range=[image_height, 0]),
            margin=dict(l=0, r=0, t=0, b=0)
        )

        # Show the figure
        return fig.show()

    except FileNotFoundError:
        print("Map file not found. Skipping...\n")

In [ ]:
# @title Visual: create_pdf_line_chart()

# Purpose:
# Create PDF chart for region visual

def create_pdf_line_chart(Main_df, map_data_path, map_name, class_colors):

    filtered_df = Main_df[Main_df['map_name'] == map_name]

    file_path = f'{map_data_path}{map_name}.jpg'

    with open(file_path, 'rb') as image_file:
        encoded_image = base64.b64encode(image_file.read()).decode()

    # Get the image dimensions
    with Image.open(image_file.name) as img:
        image_width, image_height = img.size

    try:
        # Create a line chart for the PDF of death level by class
        line_chart_data = filtered_df.groupby(['class_name', 'level']).size().reset_index(name='count')
        line_chart_data['PDF'] = line_chart_data.groupby('class_name')['count'].apply(lambda x: x / x.sum() * 100)

        # Smooth the PDF curves using KDE
        kde_line_chart_data = pd.DataFrame()
        for class_name, group in line_chart_data.groupby('class_name'):
            kde = gaussian_kde(group['level'])
            x_vals = np.linspace(group['level'].min(), group['level'].max(), num=100)
            kde_vals = kde(x_vals)
            kde_group = pd.DataFrame({'level': x_vals, 'PDF': kde_vals, 'class_name': class_name})
            kde_line_chart_data = kde_line_chart_data.append(kde_group)

        # Create line traces for each class
        line_traces = []
        for class_name, color in class_colors.items():
            class_data = kde_line_chart_data[kde_line_chart_data['class_name'] == class_name]
            line_trace = go.Scatter(
                x=class_data['level'],
                y=class_data['PDF'],
                mode='lines',
                name=class_name,
                line=dict(color=color)
            )
            line_traces.append(line_trace)

        # Create the line chart figure
        line_chart_fig = go.Figure(data=line_traces)

        # Configure the layout for the line chart figure
        line_chart_fig.update_layout(
            xaxis=dict(title='Death Level'),
            yaxis=dict(title='PDF'),
            width=image_width,
            height=image_height,
            margin=dict(l=0, r=0, t=0, b=0)
        )

        return line_chart_fig.show()

    except Exception as e:
        print("An error occurred:", str(e))
        return None

In [ ]:
# @title Visual: create_class_table()

# Purpose:
# Create Class Data Bar Chart for Region Visual

def create_class_table(Main_df, map_data_path, map_name):

    filtered_df = Main_df[Main_df['map_name'] == map_name]

    file_path = f'{map_data_path}{map_name}.jpg'

    with open(file_path, 'rb') as image_file:
        encoded_image = base64.b64encode(image_file.read()).decode()

    # Get the image dimensions
    with Image.open(image_file.name) as img:
        image_width, image_height = img.size


    # Create bar chart data for each class
    class_data = filtered_df.groupby('class_name').agg({
        'level': ['count', 'mean']
    }).reset_index()

    class_data.columns = ['Class Name', 'Record Count', 'Average Level']
    class_data['Percentage'] = class_data['Record Count'] / len(filtered_df) * 100
    class_data['Percentage'] = class_data['Percentage'].round(2)

    # Sort the table by descending record count
    class_data = class_data.sort_values('Record Count', ascending=False)

    # Add the aggregate row "Total"
    total_record_count = class_data['Record Count'].sum()
    total_average_level = class_data['Average Level'].mean()
    total_percentage = class_data['Percentage'].astype(float).sum().round(2)
    class_data = class_data.append({'Class Name': 'Total', 'Record Count': total_record_count, 'Average Level': total_average_level, 'Percentage': str(total_percentage)}, ignore_index=True)

    # Create the table figure
    table_fig = go.Figure(data=[
        go.Table(
            header=dict(values=['Class Name', 'Record Count', 'Percentage', 'Average Level'],
                        fill_color='lightgray',
                        align='left'),
            cells=dict(values=[class_data['Class Name'],
                               class_data['Record Count'],
                               class_data['Percentage'],
                               class_data['Average Level'].apply(lambda x: round(x, 2))],
                       align='left'))
    ])

    # Configure the layout for the table figure
    table_fig.update_layout(
        title='Class Statistics',
        width=image_width,
        height=250,
        margin=dict(l=0, r=0, t=0, b=0)
    )

    return table_fig.show()

In [ ]:
# @title Visual: create_npcs_table()

# Purpose:
# Create NPC table for region visual

def create_top_npcs_table(Main_df, map_data_path, map_name):

    filtered_df = Main_df[Main_df['map_name'] == map_name]

    # Read the image file and encode it to base64
    file_path = f'{map_data_path}{map_name}.jpg'

    with open(file_path, 'rb') as image_file:
        encoded_image = base64.b64encode(image_file.read()).decode()

    # Get the image dimensions
    with Image.open(image_file.name) as img:
        image_width, image_height = img.size

    # Create a table showing the top 10 NPCs with highest records
    top_npcs = filtered_df.groupby('npc_name').agg({
        'npc_avg_level': ['count', 'mean'],
        'npc_type': 'first'
    }).reset_index()

    top_npcs.columns = ['NPC Name', 'Record Count', 'Average Level', 'Type']
    top_npcs = top_npcs.sort_values('Record Count', ascending=False).head(10)

    # Create the table figure for top NPCs
    npc_table_fig = go.Figure(data=[
        go.Table(
            header=dict(values=['NPC Name', 'Record Count', 'Average Level', 'Type'],
                        fill_color='lightgray',
                        align='left'),
            cells=dict(values=[top_npcs['NPC Name'],
                               top_npcs['Record Count'],
                               top_npcs['Average Level'].apply(lambda x: round(x, 2)),
                               top_npcs['Type']],
                       align='left'))
    ])

    # Configure the layout for the NPC table figure
    npc_table_fig.update_layout(
        title='Top 10 NPCs with Highest Records',
        width=image_width,
        height=250,
        margin=dict(l=0, r=0, t=0, b=0)
    )

    return npc_table_fig.show()

# Example usage:
# create_top_npcs_table(Main_df, map_data_path, map_name)

In [ ]:
# @title Visual: region_analysis()

def region_analysis(region_analysis_df, map_name):
    try:
        df = region_analysis_df

        # Filter the DataFrame based on the map name
        notes = df[df['Zone'] == map_name]['Notes'].values

        if len(notes) > 0:
            return notes[0]
        else:
            return None

    except Exception as e:
        print("An error occurred:", str(e))
        return None

In [ ]:
# @title Call: Create Model

# Load Data to Model
data_path = '../Data/Mapping Tables.xlsx'                                                                           # Declare data path
region_summary_path = '../Data/Regional Analysis.xlsx'                                                              # Declare region summary path
map_data_path = '../Data/Maps/'                                                                                     # Declare map data path
Instance_df, Zone_df, Class_df, Race_df, NPC_df, NPC_Data_df, Death_Data_df = load_model_data(data_path)            # Load Model Data
region_analysis_df = load_region_analysis(region_summary_path)

# Create Main Analysis Dataframe
Death_Data_df = format_Death_Data_df(Death_Data_df)                                                                 # Format Death_Data_df into pivoted dataframe
Death_Data_df = drop_na_rows(Death_Data_df, 'map_id')                                                               # Drop nan map_id rows from Death_Data_df
    # CHECK: nan_counts = Death_Data_df.isna().sum()                                                                    # Check to show nan data in Death_Data_df
Main_df = create_Main_df(Death_Data_df, Zone_df, Race_df, Class_df, NPC_df, NPC_Data_df)                            # Create Main_df by mapping secondary dataframes to Death_Data_df
Main_df = clean_Main_df(Main_df)                                                                                    # Perform data cleaning on Main_df
    # CHECK: Main_df.head(5)                                                                                            # Check to show Main_df head

# Define Class & Race Colors
class_colors = define_class_colors()                                                                                # Create class_colors dictionary
race_colors = define_race_colors()                                                                                  # Create race_colors dictionary

# Create Visual Aid Dataframes
count_table = create_count_table(Main_df)                                                                           # Create count_table Count of Deaths by Class & Level df
players_alive_by_level_df = create_players_alive_by_level_df(count_table)                                           # Create players_alive_per_level_df

In [ ]:
# @title Call: Create Race Summary Chart

race_summary_chart(Main_df, race_colors)

In [ ]:
# @title Call: Create Class Summary Chart

class_summary_chart(Main_df, class_colors)

In [ ]:
# @title Call: Create Level Summary Chart

level_summary_chart(Main_df)

In [ ]:
# @title Call: Create Zone Summary Chart

zone_summary_chart(Main_df)

In [ ]:
# @title Call: Create Top Death Sources by Class Chart

top_death_sources_by_class(Main_df, class_colors, 3)

In [ ]:
# @title Call: Create Total Deaths by Class & Level Chart

deaths_by_class_and_level_chart(count_table, class_colors)

In [ ]:
# @title Call: Create Players Alive per Level at Key Milestones Table

players_alive_by_level_df[[10, 20, 30, 40, 50, 60]]

level,10,20,30,40,50,60
class_name,,,,,,
Druid,62.50,12.96,2.40,0.67,0.21,0.0
Hunter,56.42,14.22,2.84,1.06,0.33,0.0
Mage,65.79,14.10,3.69,1.17,0.37,0.0
Paladin,59.67,13.43,3.62,1.28,0.43,0.0
Priest,59.29,14.03,4.49,1.88,0.63,0.0
Rogue,60.87,10.11,2.75,1.12,0.31,0.0
Shaman,68.67,14.51,2.36,0.96,0.34,0.0
Warlock,53.43,9.08,2.22,0.71,0.14,0.0
Warrior,61.62,11.10,3.06,1.00,0.27,0.0


In [ ]:
# @title Call: Create Pre-Clustering Heatmap Visual

pre_clustering_heatmap_visual(input("Enter the map name: "), Main_df)

Enter the map name: Westfall


In [ ]:
# @title Call: Create Regional Analysis Visuals

map_name = input("Enter the map name: ")

print("Heatmap")
create_heatmap(Main_df, map_data_path, map_name, class_colors)

print("Class Table")
create_class_table(Main_df, map_data_path, map_name)

print("Top Mobs Table")
create_top_npcs_table(Main_df, map_data_path, map_name)

print("Class Deaths by Level")
create_pdf_line_chart(Main_df, map_data_path, map_name, class_colors)

print("Interpretation")
region_analysis(region_analysis_df, map_name)

Enter the map name: Westfall
Heatmap


Class Table


Top Mobs Table


Class Deaths by Level


Interpretation


"Westfall, the second zone most Alliance players venture into, also boasts one of the highest hardcore death rates. The prevalent dangers are the Defias Trappers and Pillagers who control the farms and caves. Although cave entries contribute to fatalities, the Defias Trapper stands out as a major threat. These NPCs pioneer a mechanic that restricts retreat: the net, limiting players' escape from combat."